Iniciar o particionamento de tabela tb_historicomovimentacaoClb de acordo com a RS 615258

In [0]:
USE
DLKMUMPS
GO


SELECT TOP 10
    t.NAME AS TableName,
    s.Name AS SchemaName,
    p.rows,
    SUM(a.total_pages) * 8 AS TotalSpaceKB, 
    CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS TotalSpaceMB,
    SUM(a.used_pages) * 8 AS UsedSpaceKB, 
    CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS UsedSpaceMB, 
    (SUM(a.total_pages) - SUM(a.used_pages)) * 8 AS UnusedSpaceKB,
    CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00, 2) AS NUMERIC(36, 2)) AS UnusedSpaceMB
FROM 
    sys.tables t
INNER JOIN      
    sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN 
    sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN 
    sys.allocation_units a ON p.partition_id = a.container_id
LEFT OUTER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
WHERE 
    t.NAME NOT LIKE 'dt%' 
    AND t.is_ms_shipped = 0
    AND i.OBJECT_ID > 255 
	and t.NAME = 'tb_historicomovimentacaoClb'
    GROUP BY 
    t.Name, s.Name, p.Rows
ORDER BY 
    rows DESC, t.Name

Commands completed successfully.

Total execution time: 00:00:00.0559409

(1 row affected)

Total execution time: 00:00:00.1674515

TableName,SchemaName,rows,TotalSpaceKB,TotalSpaceMB,UsedSpaceKB,UsedSpaceMB,UnusedSpaceKB,UnusedSpaceMB
tb_historicomovimentacaoClb,dlkmumps,5247642,2416912,"2360,27",2218344,"2166,35",198568,"193,91"


In [1]:
USE
DLKMUMPS
GO
    
    declare @databasename varchar(50)
	declare @tablename varchar(100)
		
	set @databasename = 'DLKMUMPS'
	set @tablename = 'tb_historicomovimentacaoClb'

	SELECT cu.CONSTRAINT_NAME, cu.COLUMN_NAME 
	FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE cu 
	WHERE EXISTS ( SELECT tc.* FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc 
	WHERE tc.CONSTRAINT_CATALOG = @databasename 
	AND tc.TABLE_NAME = @tablename 
	AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY' 
	AND tc.CONSTRAINT_NAME = cu.CONSTRAINT_NAME )

Commands completed successfully.

Total execution time: 00:00:00.0014924

(1 row affected)

Total execution time: 00:00:00.2408501

CONSTRAINT_NAME,COLUMN_NAME
PK_tb_historicomovimentacaoClb,id_historicomovimentacao


-- Retirar a tabela que faz parte do CDC -- Como Verificar se a tabela está no CDC

In [5]:
SELECT s.name AS Schema_Name, tb.name AS Table_Name
, tb.object_id, tb.type, tb.type_desc, tb.is_tracked_by_cdc
FROM sys.tables tb
INNER JOIN sys.schemas s on s.schema_id = tb.schema_id
WHERE tb.is_tracked_by_cdc = 1
and s.name = 'dlkmumps'
and tb.name = 'tb_historicomovimentacaoClb'

(1 row affected)

Total execution time: 00:00:02.2038010

Schema_Name,Table_Name,object_id,type,type_desc,is_tracked_by_cdc
dlkmumps,tb_historicomovimentacaoClb,1749581271,U,USER_TABLE,1


-- Retirar a Tabela do processo de CDC

In [6]:
-- Verificar tabela com  CDC

USE
DLKMUMPS
GO

--EXEC sys.sp_cdc_help_change_data_capture
--GO
 
SELECT capture_instance as NomeTabelaCDC, OBJECT_NAME(source_object_id) as NomeTabelaDataSource, 


LEFT(
        capture_instance, 
        CHARINDEX('_', capture_instance) -1        
    ) schema_production
    
FROM cdc.change_tables
where OBJECT_NAME(source_object_id) = 'tb_historicomovimentacaoClb'


Commands completed successfully.

Total execution time: 00:00:00.0026538

(1 row affected)

Total execution time: 00:00:00.0327294

NomeTabelaCDC,NomeTabelaDataSource,schema_production
dlkmumps_tb_historicomovimentacaoClb,tb_historicomovimentacaoClb,dlkmumps


-- Retirar a Tabela do processo de CDC

In [7]:
use
DLKMUMPS
GO

--Remover tabela com CDC
EXEC sys.sp_cdc_disable_table
    @source_schema = 'dlkmumps', -- sysname
    @source_name = 'tb_historicomovimentacaoClb', -- sysname
    @capture_instance = 'dlkmumps_tb_historicomovimentacaoClb' -- sysname



Commands completed successfully.

Total execution time: 00:00:00.0105923

Commands completed successfully.

Total execution time: 00:00:00.2311732

--Verificar se de fato a tabela saiu do CDC

In [9]:
use
DLKMUMPS
GO

SELECT s.name AS Schema_Name, tb.name AS Table_Name
, tb.object_id, tb.type, tb.type_desc, tb.is_tracked_by_cdc
FROM sys.tables tb
INNER JOIN sys.schemas s on s.schema_id = tb.schema_id
WHERE tb.is_tracked_by_cdc = 1
and s.name = 'dlkmumps'
and tb.name = 'tb_historicomovimentacaoClb'

Commands completed successfully.

Total execution time: 00:00:00.9624246

(0 rows affected)

Total execution time: 00:00:00.0308685

Schema_Name,Table_Name,object_id,type,type_desc,is_tracked_by_cdc


-- Fazer o drop da pk na tabela tb_historicomovimentacaoClb

In [21]:
USE [DLKMUMPS]
GO

/****** Object:  Index [PK_tb_historicomovimentacaoClb]    Script Date: 07/10/2020 14:37:27 ******/
ALTER TABLE [dlkmumps].[tb_historicomovimentacaoClb] DROP CONSTRAINT [PK_tb_historicomovimentacaoClb] WITH ( ONLINE = OFF )
GO


Commands completed successfully.

Total execution time: 00:00:00.0028466

Commands completed successfully.

Total execution time: 00:00:12.1812807

Validar a existência de indices não cluster na tabela que vai ser particionada.

In [10]:
/*****************************************************************************
MIT License, http://www.opensource.org/licenses/mit-license.php
Contact: help@sqlworkbooks.com
Copyright (c) 2018 SQL Workbooks LLC
Permission is hereby granted, free of charge, to any person 
obtaining a copy of this software and associated documentation
files (the "Software"), to deal in the Software without 
restriction, including without limitation the rights to use,
copy, modify, merge, publish, distribute, sublicense, and/or
sell copies of the Software, and to permit persons to whom 
the Software is furnished to do so, subject to the following 
conditions:
The above copyright notice and this permission notice shall be
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, 
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING 
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR 
OTHER DEALINGS IN THE SOFTWARE.
*****************************************************************************/
USE
DLKMUMPS
GO


declare @tablename varchar(50)
declare @schemaname varchar(50)

set @tablename = 'tb_historicomovimentacaoClb'
set @schemaname = 'dlkmumps'

SELECT 
    DB_NAME() AS database_name,
    sc.name + N'.' + t.name AS table_name,
    (SELECT MAX(user_reads) 
        FROM (VALUES (last_user_seek), (last_user_scan), (last_user_lookup)) AS value(user_reads)) AS last_user_read,
    last_user_update,
    CASE si.index_id WHEN 0 THEN N'/* No create statement (Heap) */'
    ELSE 
        CASE is_primary_key WHEN 1 THEN
            N'ALTER TABLE ' + QUOTENAME(sc.name) + N'.' + QUOTENAME(t.name) + N' ADD CONSTRAINT ' + QUOTENAME(si.name) + N' PRIMARY KEY ' +
                CASE WHEN si.index_id > 1 THEN N'NON' ELSE N'' END + N'CLUSTERED '
            ELSE N'CREATE ' + 
                CASE WHEN si.is_unique = 1 then N'UNIQUE ' ELSE N'' END +
                CASE WHEN si.index_id > 1 THEN N'NON' ELSE N'' END + N'CLUSTERED ' +
                N'INDEX ' + QUOTENAME(si.name) + N' ON ' + QUOTENAME(sc.name) + N'.' + QUOTENAME(t.name) + N' '
        END +
        /* key def */ N'(' + key_definition + N')' +
        /* includes */ CASE WHEN include_definition IS NOT NULL THEN 
            N' INCLUDE (' + include_definition + N')'
            ELSE N''
        END +
        /* filters */ CASE WHEN filter_definition IS NOT NULL THEN 
            N' WHERE ' + filter_definition ELSE N''
        END +
        /* with clause - compression goes here */
        CASE WHEN row_compression_partition_list IS NOT NULL OR page_compression_partition_list IS NOT NULL 
            THEN N' WITH (' +
                CASE WHEN row_compression_partition_list IS NOT NULL THEN
                    N'DATA_COMPRESSION = ROW ' + CASE WHEN psc.name IS NULL THEN N'' ELSE + N' ON PARTITIONS (' + row_compression_partition_list + N')' END
                ELSE N'' END +
                CASE WHEN row_compression_partition_list IS NOT NULL AND page_compression_partition_list IS NOT NULL THEN N', ' ELSE N'' END +
                CASE WHEN page_compression_partition_list IS NOT NULL THEN
                    N'DATA_COMPRESSION = PAGE ' + CASE WHEN psc.name IS NULL THEN N'' ELSE + N' ON PARTITIONS (' + page_compression_partition_list + N')' END
                ELSE N'' END
            + N')'
            ELSE N''
        END +
        /* ON where? filegroup? partition scheme? */
        ' ON ' + CASE WHEN psc.name is null 
            THEN ISNULL(QUOTENAME(fg.name),N'')
            ELSE psc.name + N' (' + partitioning_column.column_name + N')' 
            END
        + N';'
    END AS index_create_statement,
    si.index_id,
    si.name AS index_name,
    partition_sums.reserved_in_row_GB,
    partition_sums.reserved_LOB_GB,
    partition_sums.row_count,
    stat.user_seeks,
    stat.user_scans,
    stat.user_lookups,
    user_updates AS queries_that_modified,
    partition_sums.partition_count,
    si.allow_page_locks,
    si.allow_row_locks,
    si.is_hypothetical,
    si.has_filter,
    si.fill_factor,
    si.is_unique,
    ISNULL(pf.name, '/* Not partitioned */') AS partition_function,
    ISNULL(psc.name, fg.name) AS partition_scheme_or_filegroup,
    t.create_date AS table_created_date,
    t.modify_date AS table_modify_date
FROM sys.indexes AS si
JOIN sys.tables AS t ON si.object_id=t.object_id
JOIN sys.schemas AS sc ON t.schema_id=sc.schema_id
LEFT JOIN sys.dm_db_index_usage_stats AS stat ON 
    stat.database_id = DB_ID() 
    and si.object_id=stat.object_id 
    and si.index_id=stat.index_id
LEFT JOIN sys.partition_schemes AS psc ON si.data_space_id=psc.data_space_id
LEFT JOIN sys.partition_functions AS pf ON psc.function_id=pf.function_id
LEFT JOIN sys.filegroups AS fg ON si.data_space_id=fg.data_space_id
/* Key list */ OUTER APPLY ( SELECT STUFF (
    (SELECT N', ' + QUOTENAME(c.name) +
        CASE ic.is_descending_key WHEN 1 then N' DESC' ELSE N'' END
    FROM sys.index_columns AS ic 
    JOIN sys.columns AS c ON 
        ic.column_id=c.column_id  
        and ic.object_id=c.object_id
    WHERE ic.object_id = si.object_id
        and ic.index_id=si.index_id
        and ic.key_ordinal > 0
    ORDER BY ic.key_ordinal FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'),1,2,'')) AS keys ( key_definition )
/* Partitioning Ordinal */ OUTER APPLY (
    SELECT MAX(QUOTENAME(c.name)) AS column_name
    FROM sys.index_columns AS ic 
    JOIN sys.columns AS c ON 
        ic.column_id=c.column_id  
        and ic.object_id=c.object_id
    WHERE ic.object_id = si.object_id
        and ic.index_id=si.index_id
        and ic.partition_ordinal = 1) AS partitioning_column
/* Include list */ OUTER APPLY ( SELECT STUFF (
    (SELECT N', ' + QUOTENAME(c.name)
    FROM sys.index_columns AS ic 
    JOIN sys.columns AS c ON 
        ic.column_id=c.column_id  
        and ic.object_id=c.object_id
    WHERE ic.object_id = si.object_id
        and ic.index_id=si.index_id
        and ic.is_included_column = 1
    ORDER BY c.name FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'),1,2,'')) AS includes ( include_definition )
/* Partitions */ OUTER APPLY ( 
    SELECT 
        COUNT(*) AS partition_count,
        CAST(SUM(ps.in_row_reserved_page_count)*8./1024./1024. AS NUMERIC(32,1)) AS reserved_in_row_GB,
        CAST(SUM(ps.lob_reserved_page_count)*8./1024./1024. AS NUMERIC(32,1)) AS reserved_LOB_GB,
        SUM(ps.row_count) AS row_count
    FROM sys.partitions AS p
    JOIN sys.dm_db_partition_stats AS ps ON
        p.partition_id=ps.partition_id
    WHERE p.object_id = si.object_id
        and p.index_id=si.index_id
    ) AS partition_sums
/* row compression list by partition */ OUTER APPLY ( SELECT STUFF (
    (SELECT N', ' + CAST(p.partition_number AS VARCHAR(32))
    FROM sys.partitions AS p
    WHERE p.object_id = si.object_id
        and p.index_id=si.index_id
        and p.data_compression = 1
    ORDER BY p.partition_number FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'),1,2,'')) AS row_compression_clause ( row_compression_partition_list )
/* data compression list by partition */ OUTER APPLY ( SELECT STUFF (
    (SELECT N', ' + CAST(p.partition_number AS VARCHAR(32))
    FROM sys.partitions AS p
    WHERE p.object_id = si.object_id
        and p.index_id=si.index_id
        and p.data_compression = 2
    ORDER BY p.partition_number FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'),1,2,'')) AS page_compression_clause ( page_compression_partition_list )
WHERE 
    si.type IN (0,1,2) /* heap, clustered, nonclustered */
	and t.name = @tablename
	and sc.name = @schemaname
ORDER BY table_name, si.index_id
    OPTION (RECOMPILE);
GO

Commands completed successfully.

Total execution time: 00:00:00.0013142

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:01.0065674

database_name,table_name,last_user_read,last_user_update,index_create_statement,index_id,index_name,reserved_in_row_GB,reserved_LOB_GB,row_count,user_seeks,user_scans,user_lookups,queries_that_modified,partition_count,allow_page_locks,allow_row_locks,is_hypothetical,has_filter,fill_factor,is_unique,partition_function,partition_scheme_or_filegroup,table_created_date,table_modify_date
DLKMUMPS,dlkmumps.tb_historicomovimentacaoClb,2020-10-06 14:11:40.250,NULL,ALTER TABLE [dlkmumps].[tb_historicomovimentacaoClb] ADD CONSTRAINT [PK_tb_historicomovimentacaoClb] PRIMARY KEY CLUSTERED ([id_historicomovimentacao]) ON [PRIMARY];,1,PK_tb_historicomovimentacaoClb,"2,3","0,0",5247642,0,1,0,0,1,1,1,0,0,0,1,/* Not partitioned */,PRIMARY,2020-09-22 19:25:38.330,2020-10-07 14:30:14.500


-- Verificar a tipagem do campo da coluna que fará parte da Função de Partição.


In [16]:
USE
DLKMUMPS
GO

SELECT @@SERVERNAME as ServidorAzure, DB_NAME() as DatabaseName,
TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,TABLE_SCHEMA
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME  = 'tb_historicomovimentacaoClb'
and TABLE_SCHEMA = 'dlkmumps'
GO




Commands completed successfully.

Total execution time: 00:00:00.0027540

(16 rows affected)

Total execution time: 00:00:00.0590932

ServidorAzure,DatabaseName,TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,TABLE_SCHEMA
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,id_historicomovimentacao,int,NULL,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,dt_ultimoPagamento_historicomovimentacao,date,NULL,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,dt_vencultEndosso_historicomovimentacao,date,NULL,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,status_ultEndosso_historicomovimentacao,nchar,150,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,vl_premio_historicomovimentacao,decimal,NULL,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,num_qtddiasVencidos_historicomovimentacao,int,NULL,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,num_qtdParcelasaberto_historicomovimentacao,int,NULL,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,num_certificado_historicomovimentacao,nchar,25,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,num_qtdParcelasPagas_historicomovimentacao,int,NULL,dlkmumps
ISMTZBDCACHEHMG,DLKMUMPS,tb_historicomovimentacaoClb,linhaDigitavel_historicomovimentacao,nvarchar,200,dlkmumps


-- Verificar as caracteristicas do objeto

In [17]:
use
DLKMUMPS
go

sp_help 'dlkmumps.tb_historicomovimentacaoClb'

Commands completed successfully.

Total execution time: 00:00:00.0092563

No foreign keys reference table 'dlkmumps.tb_historicomovimentacaoClb', or you do not have permissions on referencing tables.

No views with schema binding reference table 'dlkmumps.tb_historicomovimentacaoClb'.

Total execution time: 00:00:00.5304009

...

Identity,Seed,Increment,Not For Replication
id_historicomovimentacao,1,1,1


RowGuidCol
No rowguidcol column defined.


Data_located_on_filegroup
PRIMARY


index_name,index_description,index_keys
PK_tb_historicomovimentacaoClb,"clustered, unique, primary key located on PRIMARY",id_historicomovimentacao


constraint_type,constraint_name,delete_action,update_action,status_enabled,status_for_replication,constraint_keys
PRIMARY KEY (clustered),PK_tb_historicomovimentacaoClb,(n/a),(n/a),(n/a),(n/a),id_historicomovimentacao


-- Criar a função de partição para a tabela  tb_historicomovimentacaoClb

In [18]:
USE
DLKMUMPS
GO



CREATE PARTITION FUNCTION [PF_Int_Partition_tb_historicomovimentacaoClb] (int)
AS RANGE LEFT FOR VALUES 
(
50000000,
100000000,
150000000,
200000000,
250000000,
300000000,
350000000,
400000000,
450000000
)

Commands completed successfully.

Total execution time: 00:00:00.0010656

Commands completed successfully.

Total execution time: 00:00:00.0302702

- -Criar a função de Scheme para a tabela tb_historicomovimentacaoClb

In [19]:
USE
DLKMUMPS
GO

CREATE PARTITION SCHEME [PS_Int_Partition_tb_historicomovimentacaoClb]
AS PARTITION [PF_Int_Partition_tb_historicomovimentacaoClb]
TO
(
    [FG_Particionamento_01],
    [FG_Particionamento_02],
    [FG_Particionamento_03],
    [FG_Particionamento_04],
    [FG_Particionamento_05],
    [FG_Particionamento_06],
    [FG_Particionamento_07],
    [FG_Particionamento_08],
    [FG_Particionamento_09],
    [FG_Particionamento_10]
)

Commands completed successfully.

Total execution time: 00:00:00.0097724

Commands completed successfully.

Total execution time: 00:00:00.0331569

-- Criação da PK com Indice Cluster já na função de partição

-- Atenção para a mudança do Partition Schema que deve ser criado, bem como o nome do campo.

In [22]:
USE
DLKMUMPS
go

/****** Object:  Index [PK_tb_historicomovimentacaoClb]    Script Date: 07/10/2020 14:37:27 ******/
ALTER TABLE [dlkmumps].[tb_historicomovimentacaoClb] ADD  CONSTRAINT [PK_tb_historicomovimentacaoClb] PRIMARY KEY CLUSTERED 
(
	[id_historicomovimentacao] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) 

ON [PS_Int_Partition_tb_historicomovimentacaoClb] (id_historicomovimentacao)


GO




Commands completed successfully.

Total execution time: 00:00:00.0046338

Commands completed successfully.

Total execution time: 00:00:40.4493448

-- Habilitar as estatisticas incrementais na tabela com o nome indice

In [23]:
USE
DLKMUMPS
GO
ALTER INDEX PK_tb_historicomovimentacaoClb ON [dlkmumps].[tb_historicomovimentacaoClb] REBUILD WITH (STATISTICS_INCREMENTAL=ON)

Commands completed successfully.

Total execution time: 00:00:00.0018930

-- Verificar com que está as fronteiras dos indices

In [25]:
use
DLKMUMPS
GO


DECLARE @tablename varchar(50)
set @tablename = 'tb_historicomovimentacaoClb'

DECLARE @schemaname varchar(50)

set @schemaname = 'dlkmumps'

SELECT s.NAME AS 'schema'
       , o.NAME AS 'table'
       , CASE o.type
             WHEN 'v' THEN 'View'
             WHEN 'u' THEN 'Table'
             ELSE o.type
         END AS objecttype
       , i.NAME AS indexname
       , i.type_desc
       , p.data_compression_desc
       , ds.type_desc AS DataSpaceTypeDesc
       , p.partition_number
       , pf.NAME AS pf_name
       , ps.NAME AS ps_name
       , CASE
             WHEN partitionds.NAME IS NULL THEN ds.NAME
             ELSE partitionds.NAME
         END AS partition_fg
       , i.is_primary_key
       , i.is_unique
       , p.rows
FROM   sys.indexes i
       INNER JOIN sys.objects o
               ON o.object_id = i.object_id
       INNER JOIN sys.data_spaces ds
               ON DS.data_space_id = i.data_space_id
       LEFT JOIN sys.schemas s
              ON o.schema_id = s.schema_id
       LEFT JOIN sys.partitions p
              ON i.index_id  = p.index_id
             AND i.object_id = p.object_id
       LEFT JOIN sys.destination_data_spaces dds
              ON i.data_space_id    = dds.partition_scheme_id
             AND p.partition_number = dds.destination_id
       LEFT JOIN sys.data_spaces partitionds
              ON dds.data_space_id = partitionds.data_space_id
       LEFT JOIN sys.partition_schemes AS ps
              ON dds.partition_scheme_id = ps.data_space_id
       LEFT JOIN sys.partition_functions AS pf
              ON ps.function_id = pf.function_id
WHERE  o.NAME = @tablename
and    s.NAME = @schemaname
ORDER  BY s.NAME
          , o.NAME
          , i.NAME
          , p.partition_number
GO

Commands completed successfully.

Total execution time: 00:00:00.0175142

(10 rows affected)

Total execution time: 00:00:00.0738655

schema,table,objecttype,indexname,type_desc,data_compression_desc,DataSpaceTypeDesc,partition_number,pf_name,ps_name,partition_fg,is_primary_key,is_unique,rows
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,1,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_01,1,1,5247642
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,2,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_02,1,1,0
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,3,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_03,1,1,0
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,4,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_04,1,1,0
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,5,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_05,1,1,0
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,6,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_06,1,1,0
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,7,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_07,1,1,0
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,8,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_08,1,1,0
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,9,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_09,1,1,0
dlkmumps,tb_historicomovimentacaoClb,Table,PK_tb_historicomovimentacaoClb,CLUSTERED,NONE,PARTITION_SCHEME,10,PF_Int_Partition_tb_historicomovimentacaoClb,PS_Int_Partition_tb_historicomovimentacaoClb,FG_Particionamento_10,1,1,0


-- Voltar com a tabela original para o processo de CDC

-- Referência importante, pois em situações onde for usada o partition Switch, Merge, etc, pode acarretar em inconsistências de dados sendo necessário refazer o CDC.

https://docs.microsoft.com/pt-br/sql/relational-databases/system-stored-procedures/sys-sp-cdc-enable-table-transact-sql?view=sql-server-ver15

In [26]:
USE
DLKMUMPS
GO

EXEC sys.sp_cdc_enable_table  
@source_schema = N'dlkmumps',  
@source_name   = N'tb_historicomovimentacaoClb',  
@role_name     = NULL,
@supports_net_changes = 1  


Commands completed successfully.

Total execution time: 00:00:00.0046892

Warning: The @allow_partition_switch parameter is set to 1. Change data capture will not track changes introduced in the table resulting from a partition switch which will cause data inconsistency when changes are consumed. Refer to books online for more information about partition switching behavior when using Change Data Capture.

Total execution time: 00:00:01.4109975

-- Query para avaliar o partition elimination

In [0]:
select id_historicomovimentacao from DLKMUMPS.dlkmumps.tb_historicomovimentacaoClb
WHERE id_historicomovimentacao = 80000000